# Computational Methods in Economics

## Problem Set - Solving Systems of Linear Equations: Suggested Solutions

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2017-11-24 13:51:29.082321


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

import numpy as np
import scipy.optimize
import scipy.linalg


# import sys
from importlib import reload

## Question 1 (A)

(a) Find the largest positive integer $n$ such that $2^n - 1$ is a DP number.

(b) Show that the distance between two adjacent DP numbers $y_1$ and $y_2$ with $y_1 < y_2$ that, 

\begin{equation}
   | y_2 - y_1 | = \epsilon_{DP} 2^{e(y_1) - 1023}. 
\end{equation}

Hint: Consider two cases for $e$ and $f$ that make two DP numbers adjacent.


## Question 2 (N)

Write a function **backward_sub** that implements the backward-substitution algorithm to solve an upper triangular system of equations in Python. As a first step, derive an expression for $x_i$, analogous to the case of forward-substitution in the lecture.

In [3]:
def backward_sub(A, b):
    """
    Implements the backward-substitution algorithm to solve an upper triangular system of equations
    
    For doctest:
    
    >>> backward_sub( np.array([[1, 1], [0, 1]]), np.array([2, 1])  )
    array([ 1.,  1.])
    
    >>> backward_sub( np.array([[1, 2, 3], [0, 5, 7], [0, 0, 9]]), np.array([1, 2, 3])  )
    array([ 0.13333333, -0.06666667,  0.33333333])
    
     
    """
    ## check input: is A a square matrix?
    n, m = A.shape
    assert n == m, "A must be a square matrix"
    
    ## initialize solution vector
    x = np.zeros(n)
    
    ## fill solution vector using a for loop
    for i in range(n):
        
        ## compute sum on numerator of recursive rule
        summ = 0
        for j in range(i):
            summ += A[(n-1)-i, (n-1)-j] * x[(n-1)-j]
        
        ## use rule; NB: start at the last element in x!
        x[(n-1)-i] = (b[(n-1)-i] - summ) / A[(n-1)-i, (n-1)-i]
        
    return x        

In [4]:
## alternative: use vector multiplication for sum (instead of inner loop)

def backward_sub2(A, b):
    """
    Implements the backward-substitution algorithm to solve an upper triangular system of equations
    
    For doctest:
    
    >>> backward_sub( np.array([[1, 1], [0, 1]]), np.array([2, 1])  )
    array([ 1.,  1.])
    
    >>> backward_sub( np.array([[1, 2, 3], [0, 5, 7], [0, 0, 9]]), np.array([1, 2, 3])  )
    array([ 0.13333333, -0.06666667,  0.33333333])
    
    
    """
    ## check input: is A a square matrix?
    n, m = A.shape
    assert n == m, "A must be a square matrix"
    
    ## initialize solution vector
    x = np.zeros(n)
    
    ## fill solution vector using a for loop
    for i in range(n):
        
        ## compute sum on numerator of recursive rule 
        summ = A[(n-1)-i, (n-1) - (i-1):] @ x[(n-1) - (i-1):]
        
        ## use rule; NB: start at the last element in x!
        x[(n-1)-i] = (b[(n-1)-i] - summ) / A[(n-1)-i, (n-1)-i]
        
    return x
        

In [5]:
## test case
A = np.array([[1, 2, 3],
              [0, 5, 7],
              [0, 0, 9]])

b = np.array([1, 2, 3])

In [6]:
backward_sub(A, b)

array([ 0.13333333, -0.06666667,  0.33333333])

In [7]:
backward_sub2(A, b)

array([ 0.13333333, -0.06666667,  0.33333333])

In [8]:
## check our result
np.linalg.solve(A, b)

array([ 0.13333333, -0.06666667,  0.33333333])

In [9]:
## Alternative: use doctest
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

## Question 3 (A)

In Miranda & Fackler, p. 11, you find an example for Gaussian elimination, specifically for the matrix:

In [10]:
A = np.array([[2, 0, -1, 2],
              [4, 2, -1, 4],
              [2, -2, -2, 3],
              [-2, 2, 7, -3]])

In their implementation of LU factorization (which is difference from ours!) they find

In [11]:
pl = np.array([[1, 0, 0, 0],
              [2, 1, 0, 0],
              [1, -1, 0, 1],
              [-1, 1, 1, 0]])
u = np.array([[2, 0, -1, 2],
              [0, 2, 1, 0],
              [0, 0, 5, -1],
              [0, 0, 0, 1]])

It is easy to verify that this is a valid LU factorization:

In [12]:
np.allclose(pl @ u, A)

True

Using Scipy's **linalg.lu** function, we get different matrices for **pl** and **u**:

In [13]:
pl, u = scipy.linalg.lu(A, permute_l=True)
print( np.allclose(pl @ u, A) )
pl, u

True


(array([[  5.00000000e-01,   3.33333333e-01,  -5.55111512e-18,
           1.00000000e+00],
        [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  5.00000000e-01,   1.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [ -5.00000000e-01,  -1.00000000e+00,   1.00000000e+00,
           0.00000000e+00]]),
 array([[ 4.        ,  2.        , -1.        ,  4.        ],
        [ 0.        , -3.        , -1.5       ,  1.        ],
        [ 0.        ,  0.        ,  5.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , -0.33333333]]))

One important takeaway from this is that there multiple ways to factor a matrix into LU components, and hence multiple valid LU representations. Use our PA = LU factorization algorithm from the lecture on matrix $A$, i.e. find $pl$ and $u$ analytically.

## Question 4 (N)

Solve Exercise 2.2 in Miranda and Fackler. That is, find the solution to a SLE with

In [15]:
A = np.array([[54, 14, -11, 2], 
              [14, 50, -4, 29],
              [-11, -4, 55, 22],
              [2, 29, 22, 95]]
            )
b = np.array([1, 1, 1, 1])

numerically, using
a) LU factorization,
b) Gauss-Jacobi,
c) Gauss-Seidel.

For LU factorization, do not use the **linalg.solve** functions in Numpy or Scipy. However, you can use Scipy's **linalg.lu** function.

## Question 5 (A)

Show that the relativ absolute difference between a real number $z$ and the DP number closest to it, $z_{DP}$, is bounded above by halt machine epsilon, i.e.
\begin{equation}
   \left| \frac{z_{DP} - z}{z} \right| \le \frac{1}{2} \epsilon_{DP}.
\end{equation}

Hint: Use what you have showed in question 1, namely that
\begin{equation}
   | y_2 - y_1 | = \epsilon_{DP} 2^{e(y_1) - 1023}, 
\end{equation}
where $y_1$ and $y_2$ are DP numbers.

## Question 6 (N)


From Judd (1998), chapter 3. Suppose that demand for good $i$ is 

\begin{equation}
    d_i(p) = a_i \sum_{j \neq i} p_j - b_ip_i + c_i,\ \ i = 1, ..., n
\end{equation}

where $a_i > b_i > 0$, and that supply is 

\begin{equation}
    s_i(p) = A_i + B_i p_i, \ \ i = 1, ..., n 
\end{equation}

where $B_i > 0$. Write a program to solve for equilibrium with sensible choices of $a_i$, $b_i$, $A_i$ and $B_i$. That means read in the parameters, check that they are consistent with commonsense economics (downward sloping demand curves, increasing supply curves and concave utility) and output the equilibrium prices and outputs. Your code should work for any number $n \ge 1$.